In [ ]:
import os
import openai
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

# Set the environment
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT")
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER")
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX")
AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
AZURE_OPENAI_CHATGPT_MODEL = os.environ.get("AZURE_OPENAI_CHATGPT_MODEL")
AZURE_OPENAI_EMB_DEPLOYMENT = os.environ.get("AZURE_OPENAI_EMB_DEPLOYMENT")

KB_FIELDS = [
    "Fields" #all Fields separated by commas from your Cog Search Index - be sure this matches those fields
]

# Set Azure Credentials - User for local & Managed Identity for Deployment
azure_credential = DefaultAzureCredential(exclude_shared_token_cache_credential = True)

# Set OpenAI details
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-05-15"

# OpenAPI credentialing
openai.api_type = "azure_ad"
openai.api_key = azure_credential.get_token("https://cognitiveservices.azure.com/.default").token

# Set up clients for Cognitive Search and Storage
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=azure_credential)

In [ ]:
# Chat roles
SYSTEM = "system"
USER = "user"
ASSISTANT = "GTI.ai"

system_message_chat_conversation = """
Review the Azure Demo
"""
chat_logs= [{"role" : SYSTEM, "content" : system_message_chat_conversation}]

summary_prompt_template = """
Review the Azure Demo

Search query:
"""
query_summary_conversations = [{"role" : SYSTEM, "content" : summary_prompt_template}]

In [ ]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "Add a query for your data here"
query_summary_conversations.append({"role": USER, "content": user_input })

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

query_completion = openai.ChatCompletion.create(
    deployment_id=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    messages=query_summary_conversations, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)
search = query_completion.choices[0].message.content

# Use Azure OpenAI to compute an embedding for the query
query_vector = openai.Embedding.create(engine=AZURE_OPENAI_EMB_DEPLOYMENT, input=search)["data"][0]["embedding"]

# Alternatively simply use the following if not using reranking with semantic search:
# search_client.search(search, 
#                      top=3, 
#                      vector=query_vector if query_vector else None, 
#                      top_k=50 if query_vector else None, 
#                      vector_fields="embedding" if query_vector else None)

print("Searching:", search)
print("-------------------")
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None
r = search_client.search(search, 
                         filter=filter,
                         query_type=QueryType.SEMANTIC, 
                         query_language="en-us", 
                         query_speller="lexicon", 
                         semantic_configuration_name="default", 
                         top=3,
                         # vector=query_vector if query_vector else None, 
                         # top_k=50 if query_vector else None, 
                         # vector_fields="embedding" if query_vector else None)
)
results = [str(doc['sheetName']) + " - " + str(doc['Field1']) + ": " + str(doc['Field2'])  for doc in r] # truncated. be sure to add these to match you Index fields

content = "\n".join(results)

user_content = user_input + " \nSOURCES:\n" + content

chat_logs.append({"role": USER, "content": user_content })

chat_completion = openai.ChatCompletion.create(
    deployment_id=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    messages=chat_logs, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)
chat_content = chat_completion.choices[0].message.content
'''
reset user content to avoid sources in conversation history
add source as a single shot in query conversation
'''
chat_logs[-1]["content"] = user_input
chat_logs.append({"role":ASSISTANT, "content": chat_content})
query_summary_conversations.append({"role":ASSISTANT, "content": chat_content})

print("\n-------------------\n")
for conversation in chat_logs:
    print(conversation)